# NBA Last Two Minute Reports

To increase transparency into officiating, the NBA began to publish reports that contain a breakdown of calls in the last two minutes of close games. Beginning on March 1, 2015, these reports detail all calls and notable non-calls in games that had a point differential of three or fewer at any time in the last two minutes. [This site](https://official.nba.com/2019-20-nba-officiating-last-two-minute-reports/) describes the process in more detail along with reports from this season and archived reports.

The goal of this notebook is to explore the reports and use data visualization to further enhance the takeaways from the data available on the website linked above.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Arc
import itertools
from datetime import datetime, timedelta

from PIL import Image
import PIL.ImageOps    
import time

from py_ball import image, boxscore, scoreboard

HEADERS = {'Connection': 'close',
           'Host': 'stats.nba.com',
           'Origin': 'http://stats.nba.com',
           'Upgrade-Insecure-Requests': '1',
           'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2)' + \
                         'AppleWebKit/537.36 (KHTML, like Gecko) ' + \
                         'Chrome/66.0.3359.117 Safari/537.36'}

TEAM_TO_ID_NBA = {'TOR': '1610612761', 'DEN': '1610612743',
                  'DET': '1610612765', 'NOP': '1610612740',
                  'MIL': '1610612749', 'GSW': '1610612744',
                  'SAS': '1610612759', 'POR': '1610612757',
                  'LAC': '1610612746', 'DAL': '1610612742',
                  'MEM': '1610612763', 'PHI': '1610612755',
                  'BOS': '1610612738', 'MIN': '1610612750',
                  'CHA': '1610612766', 'IND': '1610612754',
                  'ORL': '1610612753', 'MIA': '1610612748',
                  'HOU': '1610612745', 'SAC': '1610612758',
                  'UTA': '1610612762', 'BKN': '1610612751',
                  'ATL': '1610612737', 'PHX': '1610612756',
                  'WAS': '1610612764', 'NYK': '1610612752',
                  'OKC': '1610612760', 'LAL': '1610612747',
                  'CLE': '1610612739', 'CHI': '1610612741'}

pd.options.mode.chained_assignment = None  # Disabling pandas SetWithCopyWarnings

Let's read in the report data and do some exploration. These data were sourced from the above official's last-two-minute site that contains each qualifying play this season.

In [2]:
l2m_2019 = pd.read_csv('l2m.csv')
l2m_2019.head()

,Period,Time,Call Type,Committing Player,Disadvantaged Player,Review Decision,Video,Comment,game_id
0,Q4,01:53.3,Foul: Offensive,OG Anunoby,Brandon Ingram,CNC,http://official.nba.com/last-two-minute-report...,Anunoby (TOR) firms up in a legal screening po...,21900001
1,Q4,01:44,Foul: Offensive,Serge Ibaka,Nicolo Melli,CNC,http://official.nba.com/last-two-minute-report...,Ibaka (TOR) makes marginal contact with Melli ...,21900001
2,Q4,01:41.4,Foul: Shooting,JJ Redick,Kyle Lowry,CNC,http://official.nba.com/last-two-minute-report...,Redick (NOP) contests Lowry's (TOR) jump shot ...,21900001
3,Q4,01:34.5,Foul: Personal,Fred VanVleet,Brandon Ingram,CNC,http://official.nba.com/last-two-minute-report...,VanVleet (TOR) maintains a legal guarding posi...,21900001
4,Q4,01:09.6,Foul: Personal,Fred VanVleet,Brandon Ingram,INC,http://official.nba.com/last-two-minute-report...,VanVleet (TOR) is late to establish himself in...,21900001


And here's the data from the 2018 regular season after the All Star break.

In [4]:
l2m_2018 = pd.read_csv('l2m_2018.csv')
l2m_2018.head()

,Period,Time,Call Type,Committing Player,Disadvantaged Player,Review Decision,Video,Comment,game_id
0,Q4,01:58.0,Foul: Shooting,JJ Redick,Dwyane Wade,CNC,http://official.nba.com/last-two-minute-report...,Redick (PHI) makes marginal contact to the bod...,21800868
1,Q4,01:45.6,Foul: Personal,Dion Waiters,Jimmy Butler,CNC,http://official.nba.com/last-two-minute-report...,Waiters (MIA) briefly engages with Butler (PHI...,21800868
2,Q4,01:43.0,Turnover: Traveling,Ben Simmons,,CNC,http://official.nba.com/last-two-minute-report...,B. Simmons (PHI) maintains his pivot foot on t...,21800868
3,Q4,01:40.5,Foul: Offensive,Boban Marjanovic,Dion Waiters,CNC,http://official.nba.com/last-two-minute-report...,Marjanovic (PHI) makes marginal contact with W...,21800868
4,Q4,01:36.2,Foul: Shooting,Justise Winslow,Boban Marjanovic,CC,http://official.nba.com/last-two-minute-report...,Winslow (MIA) makes contact to the arm of Marj...,21800868


To bolster the sample, we can combine these datasets.

In [7]:
l2m_df = pd.concat([l2m_2018, l2m_2019], axis=0)

In [8]:
l2m_df

,Period,Time,Call Type,Committing Player,Disadvantaged Player,Review Decision,Video,Comment,game_id
0,Q4,01:58.0,Foul: Shooting,JJ Redick,Dwyane Wade,CNC,http://official.nba.com/last-two-minute-report...,Redick (PHI) makes marginal contact to the bod...,21800868
1,Q4,01:45.6,Foul: Personal,Dion Waiters,Jimmy Butler,CNC,http://official.nba.com/last-two-minute-report...,Waiters (MIA) briefly engages with Butler (PHI...,21800868
2,Q4,01:43.0,Turnover: Traveling,Ben Simmons,,CNC,http://official.nba.com/last-two-minute-report...,B. Simmons (PHI) maintains his pivot foot on t...,21800868
3,Q4,01:40.5,Foul: Offensive,Boban Marjanovic,Dion Waiters,CNC,http://official.nba.com/last-two-minute-report...,Marjanovic (PHI) makes marginal contact with W...,21800868
4,Q4,01:36.2,Foul: Shooting,Justise Winslow,Boban Marjanovic,CC,http://official.nba.com/last-two-minute-report...,Winslow (MIA) makes contact to the arm of Marj...,21800868
...,...,...,...,...,...,...,...,...,...
479,Q4,00:32.4,Foul: Personal,Eric Gordon,Bradley Beal,CNC,http://official.nba.com/last-two-minute-report...,Gordon (HOU) maintains a legal guarding positi...,21900061
480,Q4,00:13.9,Foul: Personal,Isaiah Thomas,P.J. Tucker,CC,http://official.nba.com/last-two-minute-report...,Thomas (WAS) commits a take foul on Tucker (HOU).,21900061
481,Q4,00:07.7,Foul: Shooting,P.J. Tucker,Bradley Beal,CC,http://official.nba.com/last-two-minute-report...,Replay review of the foul called on Tucker (HO...,21900061
482,Q4,00:02.4,Foul: Shooting,Isaac Bonga,James Harden,CC,http://official.nba.com/last-two-minute-report...,Bonga (WAS) makes contact to the body of Harde...,21900061


In [9]:
list(set(l2m_df['Call Type']))

[nan,
 'Turnover: Out of Bounds',
 'Violation: Jump Ball',
 'Turnover: 5 Second Inbound',
 'Turnover: 8 Second Violation',
 'Foul: Offensive Charge',
 'Turnover: Discontinue Dribble',
 'Instant Replay: Support Ruling',
 'Turnover: 5 Second Violation',
 'Turnover: Backcourt Turnover',
 'Foul: Away from Play',
 'Turnover: Lost Ball Out of Bounds',
 'Violation: Delay of Game',
 'Turnover: Kicked Ball Violation',
 'Foul: Technical',
 'Foul: Flagrant Type 1',
 'Turnover:  Out of Bounds - Bad Pass Turn',
 'Foul: Personal',
 'Turnover: 10 Second Violation',
 'Turnover: Stepped out of Bounds',
 'Turnover: Double Dribble',
 'Stoppage: Out-of-Bounds',
 'Turnover: 3 Second Violation',
 'Foul: Double Technical',
 'Turnover: 24 Second Violation',
 'Turnover: Palming',
 'Foul: Shooting',
 'Foul: Loose Ball',
 'Violation: Defensive Goaltending',
 'Instant Replay: Overturn Ruling',
 'Violation: Lane',
 'Violation: Kicked Ball',
 'Turnover: Offensive Goaltending',
 'Foul: Offensive',
 'Foul: Personal T

There are many types of calls that get reviewed in these reports, which belong to 5 classes:
- Foul
- Turnover
- Violation
- Instant Replay
- Stoppage

We can store these classes in a feature, `Call Class`.

In [11]:
l2m_df['Call Class'] = ['None' if pd.isnull(x) else
                        'Foul' if 'Foul:' in x else
                        'Turnover' if 'Turnover:' in x else
                        'Violation' if 'Violation:' in x else
                        'Instant Replay' if 'Instant Replay:' in x else
                        'Stoppage' for x in l2m_df['Call Type']]

In [14]:
l2m_df.groupby('Call Class')['Call Class'].count()

Call Class
Foul              3070
Instant Replay       8
None                 5
Stoppage             3
Turnover           310
Violation           40
Name: Call Class, dtype: int64

The vast majority of calls reviewed in the Last Two Minute reports involve foul calls or non-calls, followed by turnovers. The rest of the classifications are relatively rare. 

The NBA also abbreviates common plays with the following definitions:
- SQBR - Speed, Quickness, Balance, Rhythm
- POC - Point of Contact
- OOB - Out of Bounds
- FOM - Freedom of Movement

Let's extract these into a `Play Class` feature.

In [25]:
l2m_df['Play Class'] = ['None' if pd.isnull(x) else
                        'SQBR' if 'SQBR' in x else
                        'POC' if 'POC' in x else
                        'OOB' if 'OOB' in x else
                        'FOM' if 'FOM' in x else
                        'Other' for x in l2m_df['Comment']]

In [26]:
l2m_df.groupby('Play Class')['Play Class'].count()

Play Class
FOM        76
Other    3117
SQBR      243
Name: Play Class, dtype: int64

Even though the NBA defines several common play types, only FOM and SQBR plays make it into the comments. Let's dive deeper on fouls, the most commonly reviewed play class.

## Fouls

In [29]:
foul_df = l2m_df[l2m_df['Call Class']=='Foul']
foul_df.groupby(['Call Type', 'Review Decision'])['Call Type'].count()

Call Type               Review Decision
Foul: Away from Play    CC                   2
                        CNC                 35
                        INC                 12
Foul: Clear Path        CC                   2
Foul: Defense 3 Second                      18
                        CNC                 33
                        INC                  4
Foul: Double Technical  CC                   1
Foul: Flagrant Type 1   CC                   1
Foul: Loose Ball        CC                  30
                        CNC                278
                        IC                   1
                        INC                 27
Foul: Offensive                              3
                        CC                  16
                        CNC                721
                        IC                   1
                        INC                 38
Foul: Offensive Charge  CC                   1
Foul: Personal                               2
                    